<a href="https://colab.research.google.com/github/jenochs/video-generation-book/blob/main/notebooks/hunyuan_colab_a100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HunyuanVideo on Google Colab A100

**Generate high-quality videos using Tencent's HunyuanVideo model on Google Colab A100 GPU**

🚀 **What you'll learn:**
- Run the 13B parameter HunyuanVideo model on Colab A100 (40GB)
- Optimize memory usage for large-scale video generation
- Generate videos up to 15 seconds with advanced prompting
- Export and download high-quality video results

⚡ **Requirements:**
- Google Colab Pro+ with A100 GPU access
- ~20-30 minutes for complete setup
- Google Drive for video storage (optional)

📚 **From the Book:** *Hands-On Video Generation with AI* - Chapter 3: Advanced Model Implementation

In [ ]:
# Check GPU and configure environment
\!nvidia-smi

import os, torch
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.backends.cuda.matmul.allow_tf32 = True
print(f"✅ GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory/1024**3:.1f} GB")

In [ ]:
# Install required packages (optimized for Colab)
\!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --upgrade
\!pip install diffusers>=0.33.1 transformers>=4.52.4 --upgrade
\!pip install xformers imageio-ffmpeg accelerate safetensors --upgrade
\!pip install git+https://github.com/jenochs/video-generation-book.git

In [ ]:
# Memory monitoring utilities
import torch, gc

def get_gpu_memory():
    if not torch.cuda.is_available(): return {"total": 0, "free": 0}
    total = torch.cuda.get_device_properties(0).total_memory / 1024**3
    cached = torch.cuda.memory_reserved() / 1024**3
    return {"total": total, "free": total - cached}

def print_memory_status(stage=""):
    mem = get_gpu_memory()
    print(f"🧠 GPU Memory {stage}: {mem["free"]:.1f}GB free / {mem["total"]:.1f}GB total")

def cleanup_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

print_memory_status("(Initial)")

In [ ]:
# Load HunyuanVideo model with A100 optimizations
from diffusers import HunyuanVideoPipeline
import time

print("🔄 Loading HunyuanVideo model...")
start_time = time.time()

try:
    pipe = HunyuanVideoPipeline.from_pretrained(
        "hunyuanvideo-community/HunyuanVideo",
        torch_dtype=torch.float16,
        use_safetensors=True,
        low_cpu_mem_usage=True
    )
    
    # A100 40GB optimizations
    pipe.enable_sequential_cpu_offload()
    pipe.vae.enable_tiling()
    pipe.vae.enable_slicing()
    
    try:
        pipe.enable_xformers_memory_efficient_attention()
        print("   ✅ xFormers enabled")
    except:
        print("   ⚠️ xFormers not available")
    
    load_time = time.time() - start_time
    print(f"✅ Model loaded in {load_time:.1f}s")
    print_memory_status("(After loading)")
    
except Exception as e:
    print(f"❌ Failed to load: {e}")
    raise

In [ ]:
# Simple video generation function
from IPython.display import Video, display
import imageio, numpy as np, os
from datetime import datetime

os.makedirs("/content/videos", exist_ok=True)

def generate_video(prompt, height=544, width=960, frames=32, steps=25, seed=42):
    """Generate video optimized for A100 40GB"""
    print(f"🎬 Generating: {prompt[:50]}...")
    print(f"🎯 {width}x{height}, {frames} frames, {steps} steps")
    
    try:
        result = pipe(
            prompt=prompt,
            height=height, width=width, num_frames=frames,
            guidance_scale=6.0, num_inference_steps=steps,
            generator=torch.Generator(device="cuda").manual_seed(seed)
        )
        
        # Save video (convert PIL to numpy)
        timestamp = datetime.now().strftime("%H%M%S")
        output_path = f"/content/videos/video_{timestamp}.mp4"
        
        print("💾 Saving...")
        with imageio.get_writer(output_path, fps=8, codec="h264") as writer:
            for frame in result.frames[0]:
                frame_array = np.array(frame.convert("RGB"))
                writer.append_data(frame_array)
        
        print(f"✅ Saved: {output_path}")
        return output_path
        
    except Exception as e:
        print(f"❌ Error: {e}")
        cleanup_memory()
        raise

print("🎬 Generation function ready\!")

In [ ]:
# Generate test video
video_path = generate_video(
    "A majestic golden eagle soaring over snow-capped mountains at sunset, cinematic"
)

# Display result
display(Video(video_path, width=600))
print_memory_status("(After generation)")

## 🎉 Success\!

You've successfully:
- ✅ Loaded 13B parameter HunyuanVideo on A100
- ✅ Generated high-quality AI video
- ✅ Optimized for 40GB memory constraints

### 🚀 Try More Prompts:


**Happy video generating\! 🎬✨**